In [78]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
measurement = np.load('../test/dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(f"Training data: {trainCIR.shape}")
# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

# Set the number of dictionary atoms (components) and sparsity level
# N_components = 3  # Number of Sparse Components
n_components = 20  # Try values like 20, 50, or 100
n_nonzero_coefs = 17 # Sparsity level
no_train_cirs = trainCIR.shape[0]
no_test_cirs = testCIR.shape[0]

Training data: (800, 15, 251, 2)


In [79]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))

# Learn the dictionary
dl_alice = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', n_jobs= -1)
dl_eve = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', n_jobs= -1)

dl_alice.fit(train_alice_magnitude)
dictionary_alice = dl_alice.components_

dl_eve.fit(train_eve_magnitude)
dictionary_eve = dl_eve.components_

# Create an OMP object
omp_alice = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)  # adjust the number of non-zero coefficients as needed
omp_eve = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)  # adjust the number of non-zero

sparse_rep_alice = np.zeros((no_train_cirs, n_components))
sparse_rep_eve = np.zeros((no_train_cirs, n_components))

for i in range(no_train_cirs):
    # For Alice
    omp_alice.fit(dictionary_alice.T, train_alice_magnitude[i]) 
    sparse_rep_alice[i] = omp_alice.coef_

    # For Eve
    omp_eve.fit(dictionary_eve.T, train_eve_magnitude[i]) 
    sparse_rep_eve[i] = omp_eve.coef_
    
print(sparse_rep_alice[0])


[39783.86505443  8196.0872501   6723.87504859  2988.57045789
     0.          5191.95719631  2080.45900255 -2532.34369478
  3332.62308214 -2623.47100748  2473.17806471  6179.80922965
  1897.44449156 -2517.87993042  2683.87214466  1553.75430033
     0.             0.         -3026.36256369 -2313.65357665]


In [80]:
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]

# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))

dl_test_alice = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', n_jobs= -1)
dl_test_eve = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', n_jobs= -1)

dl_test_alice.fit(test_alice_magnitude)
dictionary_test_alice = dl_test_alice.components_

dl_test_eve.fit(test_eve_magnitude)
dictionary_test_eve = dl_test_eve.components_

# Create an OMP object
omp_test_alice = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)  # adjust the number of non-zero coefficients as needed
omp_test_eve = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)

sparse_rep_test_alice = np.zeros((no_test_cirs, n_components))
sparse_rep_test_eve = np.zeros((no_test_cirs, n_components))

for i in range(no_test_cirs):
    # For test_Alice
    omp_test_alice.fit(dictionary_test_alice.T, test_alice_magnitude[i]) 
    sparse_rep_test_alice[i] = omp_test_alice.coef_

    # For test_Eve
    omp_test_eve.fit(dictionary_test_eve.T, test_eve_magnitude[i]) 
    sparse_rep_test_eve[i] = omp_test_eve.coef_
    
print(sparse_rep_test_alice[0])

[35994.88824055 12598.57533772 -4247.88966424  3592.73914423
 -6334.63230621 -1400.54963027 -2598.10143719     0.
 -2332.8865297   1019.41664584     0.          2237.30381949
 -1565.28230188 -4003.12336365 -4316.94125435  -733.66292461
  -854.61043152  -958.58671013     0.          -478.3608271 ]


In [81]:
print(f"Training data: {sparse_rep_alice.shape}")

train_sparse_data = np.vstack((sparse_rep_alice, sparse_rep_eve))
test_sparse_data = np.vstack((sparse_rep_test_alice, sparse_rep_test_eve))

# Labels for training data
train_alice_labels = np.zeros(sparse_rep_alice.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(sparse_rep_eve.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Labels for test data
test_alice_labels = np.zeros(sparse_rep_test_alice.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(sparse_rep_test_eve.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))


Training data: (800, 20)


In [82]:
# # Dictionary
# D = train_sparse_codes.T
# print('Dictionary shape:', D.shape)

In [83]:
scaler = StandardScaler()
train_sparse_codes = scaler.fit_transform(train_sparse_data)
# Apply the same scaling to test data
test_sparse_codes = scaler.transform(test_sparse_data)
print('Train data shape:', train_sparse_codes.shape)

Train data shape: (1600, 20)


In [84]:
# Initialize and train the SVM classifier
classifier = SVC(kernel='rbf', random_state=42)
classifier.fit(train_sparse_codes, train_labels)

# Predict on test data
predictions = classifier.predict(test_sparse_codes)

In [85]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Classification Accuracy: 67.50%
tp: 130
tn: 140
fp: 60
fn: 70
MDR: 0.3
FAR: 0.35
AR: 0.675
